# Hyperparameter Tuning using HyperDrive


In [12]:
from azureml.core import Workspace, Experiment, Dataset, Datastore
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import InferenceConfig


from azureml.core import Experiment, Workspace, Environment, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Model, Run
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import BanditPolicy, BayesianParameterSampling, RandomParameterSampling, choice, uniform, loguniform
from azureml.train.hyperdrive import HyperDriveRunConfig, HyperDriveRun

from azureml.train.sklearn import SKLearn
from azureml.widgets import RunDetails

## Dataset

The same dataset using the automl experiment is going to be used for the Hyperdrive experiment.
The dataset is loaded in the train.py script and processed so it can be used as an Input to the model

In [2]:
ws = Workspace.from_config()
experiment_name = 'capstone-project'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [4]:
list(range(0.01,11))

[3, 4, 5, 6, 7, 8, 9, 10]

### setting up compute cluster

In [3]:
# Choose a name for your CPU cluster
cpu_cluster_name = "capstone-cluster2"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                            min_nodes=1,
                                                           max_nodes=6)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [27]:
# Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1)

# Create the different params that you will be using during training
param_sampling =  RandomParameterSampling({
    '--learning_rate': loguniform(-3, 0),
    '--max_depth': choice(3, 4, 5, 6, 7, 8, 9, 10),
    '--n_estimators': choice(100, 200, 300, 400, 500, 600, 700, 800, 900, 1000),
    '--reg_lambda': choice(0.001, 0.01, 0.1, 1, 10, 100),
    '--subsample': uniform(0.5, 1),
    '--colsample_bytree': uniform(0.6, 1)
})

# setting up environment for the train.py script
env = Environment(name="capstone-hyperdrive-env")

conda_dep = CondaDependencies()
conda_dep.add_conda_package("scikit-learn")
conda_dep.add_conda_package("numpy")
conda_dep.add_pip_package("pandas")
conda_dep.add_conda_package("xgboost")
conda_dep.add_pip_package("azureml-sdk")
conda_dep.add_pip_package("joblib")
env.python.conda_dependencies = conda_dep

# Creating a ScriptRunConfig instead of and Estimator, because these have been deprecated
src = ScriptRunConfig(
    source_directory='.',
    script='train.py',
    compute_target=cpu_cluster,
    environment=env
)

# Creating HyperdriveConfig
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=param_sampling,
                                     max_total_runs=1000,
                                     policy=early_termination_policy,
                                     primary_metric_name='NRMSE',
                                     primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
                                     max_duration_minutes=30,
                                     max_concurrent_runs=6)

In [28]:
# Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_config)

## Run Details


In [29]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model


In [30]:
# get best run
best_run = hyperdrive_run.get_best_run_by_primary_metric()

In [31]:
# Get the best run's metrics and hyperparameters
best_run_metrics = best_run.get_metrics()
best_run_hyperparams = best_run.get_details()['runDefinition']['arguments']

In [32]:
# Print the best run's details
print("Best Run ID:", best_run.id)
print("Best Run Metrics:", best_run_metrics)
print("Best Run Hyperparameters:", best_run_hyperparams)

Best Run ID: HD_4279bbc3-e975-43fb-bd23-4a1eeb02aa78_104
Best Run Metrics: {'Learning rate': 0.07487119019678894, 'Max depth': 10, 'Number of estimators': 700, 'L2 regularization Strength': 0.001, 'Subsample': 0.9687145955692129, 'Fraction of cols sampled by tree': 0.9789816596693053, 'NRMSE': 0.0005410834853574961}
Best Run Hyperparameters: ['--colsample_bytree', '0.9789816596693053', '--learning_rate', '0.07487119019678894', '--max_depth', '10', '--n_estimators', '700', '--reg_lambda', '0.001', '--subsample', '0.9687145955692129']


# Save the best model

In [35]:
# save model best model in the workspace
best_run.download_file('outputs/model.joblib')

# register best model
best_model = best_run.register_model(model_name='capstone_best_hyperdrive_model', model_path='outputs/model.joblib')

## Model Deployment

Comparing the best model yield by Automl and Hyperdrive jobs, We can observer the following:<br>

1. The best model from AutoML has a normalized root mean squared error of 0.05943
2. The best model from Hyperdrive has a normalized root mean squared error of 5.4108e-4

Since the fine-tuned Xgboost Regressor found by the Hyperdrive job performs the best, this is the model that will be deployed.

In [17]:
best_run.get_file_names()

['outputs/model.joblib',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

### writting score.py to be run in the ACI instance to perform inference:

In [19]:
%%writefile score.py
import json
import numpy as np
import os
import joblib
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path('outputs/model.joblib') 
    model = joblib.load(model_path)

def run(raw_data):
    data = json.loads(raw_data)['data']
    data = np.array(data)
    result = model.predict(data)

    return json.dumps({"forecast": result.tolist()})

Writing score.py


In [20]:
# retrieving best hyperdrive run environment, so we can use the same environment
# configuration during inference
best_run = Run(experiment, run_id='HD_4279bbc3-e975-43fb-bd23-4a1eeb02aa78_104')
env=best_run.get_environment()

In [21]:
# Create an inference configuration object
inference_config = InferenceConfig(entry_script="score.py", environment=env)
# defining deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1,
                                                        tags={'type': 'xgboot regressor'},
                                                        description='Regressor that predicts the next 4 weeks of sales given historical data')

In [22]:
# retrieving best model from hyperdrive experiment
model = Model(ws, 'capstone_best_hyperdrive_model')

In [23]:
service_name = 'capstone-web-service'
aci_service = Model.deploy(ws, service_name, [model],
                        inference_config=inference_config,
                        deployment_config=deployment_config)
                        
aci_service.wait_for_deployment(show_output=True)

azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-08-02 15:08:32+00:00 Creating Container Registry if not exists.
2023-08-02 15:08:32+00:00 Registering the environment.
2023-08-02 15:08:34+00:00 Use the existing image.
2023-08-02 15:08:34+00:00 Generating deployment configuration.
2023-08-02 15:08:35+00:00 Submitting deployment to compute.
2023-08-02 15:08:42+00:00 Checking the status of deployment capstone-web-service.

In [ ]:
# getting some data to test the endpoint


TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

